# Generating a random fermionic guassian statevector (Method 1)

In [1]:
using Pkg
Pkg.add("StatsBase")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Constructing rho

In [ ]:
using LinearAlgebra


# Generating a random Hermitian matrix (A)
function random_hermitian(N)
    A = randn(ComplexF64, N, N)
    H = (A + A') / 2
    return H
end
A = random_hermitian(1024)


# Generating a random skew symmetric matrix (B)
function random_skew_symmetric_complex(N)
    A = randn(ComplexF64, N, N)
    return (A - transpose(A)) / 2
end
B  = random_skew_symmetric_complex(1024)



# Constructing Hamiltonian based on A and B
function construct_H(A::AbstractMatrix, B::AbstractMatrix)
    n, m = size(A)
    @assert size(B) == (n, m) "A and B must be the same size"

    H = Matrix{eltype(A)}(undef, 2n, 2m)

    # Fill in blocks directly
    H[1:n, 1:m]     .= -conj.(A)
    H[1:n, m+1:2m]  .= B
    H[n+1:2n, 1:m]  .= -conj.(B)
    H[n+1:2n, m+1:2m] .= A

    return H
end


H = construct_H(A, B)

In [ ]:
#Checking if the generated matrix is Hermitian
ishermitian(H)

true

In [ ]:
# Constructing rho
function build_rho(H::Matrix{ComplexF64})
    exp_H = exp(-H)           
    Z = tr(exp_H)                
    rho = exp_H / Z             
    return rho
end
rho = build_rho(H)


2048×2048 Matrix{ComplexF64}:
  0.000238328+0.0im          …    -6.2024e-6-0.000221598im
   5.00221e-5-0.00014411im       -8.62595e-5-3.86008e-5im
  -4.36494e-5-4.995e-5im          5.70648e-5+0.000332016im
   1.75094e-5+0.00017093im        0.00047329-0.000183868im
   9.90781e-6-0.00010147im      -0.000170651+2.86602e-5im
   4.57028e-5-0.00018719im   …  -0.000225226-0.000157629im
 -0.000126374-9.14804e-5im       -9.22026e-5+0.000111875im
  -5.24248e-5+9.47367e-5im       0.000134278+0.000225361im
  0.000172677+4.26422e-5im        9.10329e-5-0.000484498im
    6.6121e-5-4.93722e-5im      -0.000246242+0.000142495im
             ⋮               ⋱  
  -2.76319e-5+0.000132661im      -3.59745e-5-6.49509e-6im
   5.66139e-6+5.06112e-6im   …    8.29844e-5-0.000184052im
  0.000129886-9.37108e-5im       -0.00013448-0.000284161im
  0.000165793+5.94278e-5im      -0.000139871-0.000281329im
  -8.26815e-5+1.0517e-5im         7.23226e-5+0.000132971im
 -0.000136026-0.000126813im      -3.95192e-5+0.00025269

## Making rho a pure state in a larger Hilbert space

In [ ]:
# Making rho a pure state in a larger Hilbert space
function purification_rho(rho::Matrix{ComplexF64})
    vals, vecs = eigen(rho)  # rho = vecs * Diagonal(vals) * vecs'

    d = size(rho, 1)
    psi = zeros(ComplexF64, d^2)  # Purified state vector lives in d x d space

    for k in 1:d
        pk = vals[k]
        if pk ≈ 0
            continue  # Skip zero-eigenvalue components
        end
        psi_k = vecs[:, k]
        tr_k = zeros(ComplexF64, d)
        tr_k[k] = 1.0  # |k⟩ in ancilla space


        psi += sqrt(pk) * kron(psi_k, tr_k)
    end

    # Normalize Ψ (should already be normalized, but numerically might not be)
    psi ./= norm(psi)

    return psi
end
statevector = purification_rho(rho)



# Saving the statevector to a text file
open("FGS_Gen.txt", "w") do file
    for i in 1:length(statevector)
        real_part = real(statevector[i])
        imag_part = imag(statevector[i])
        write(file, "$real_part + $imag_part im\n")
    end
end

DomainError: DomainError with -2.108425289312159e-16:
sqrt was called with a negative real argument but will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

## Sample from the density matrix

In [ ]:
using LinearAlgebra
using StatsBase

# Sample from the density matrix
function sample_from_rho(rho::Matrix{ComplexF64})
    vals, vecs = eigen(Hermitian(rho))  # ρ = vecs * Diagonal(vals) * vecs'

    probs = abs.(vals) ./ sum(abs.(vals))

    # Sample index k with probability λ_k
    k = sample(1:length(probs), Weights(probs))

    ψ = vecs[:, k]
    ψ ./= norm(ψ)
    return ψ
end

SVEC = sample_from_rho(rho)

# Saving the sampled statevector to a text file
open("FGS_Gen_2.txt", "w") do file
    for i in 1:length(SVEC)
        real_part = real(SVEC[i])
        imag_part = imag(SVEC[i])
        write(file, "$real_part + $imag_part im\n")
    end
end


sample_from_rho (generic function with 1 method)

# Generating a random fermionic guassian statevector (Method 2)

In [ ]:
using LinearAlgebra
using Random

# Generating a random quadratic Hamiltonian
function random_quadratic_H(N::Int)
    A = randn(ComplexF64, N, N)
    H = A + A'  # Make it Hermitian
    return H
end

# Diagonalizing the Hamiltonian
function diagonalize_H(H::Matrix{ComplexF64})
    eig = eigen(H)
    E = eig.values
    U = eig.vectors  # Columns are eigenvectors
    return E, U
end


# Generating a random fermionic guassian statevector(Through slater determinant)
function random_fgs(U::Matrix{ComplexF64}, Nf::Int)
    L = size(U, 1)
    chosen = sort(randperm(L)[1:Nf])  # choose random orbitals
    psi = zeros(ComplexF64, 2^L)

    for i in 0:(2^L - 1)
        bits = digits(i, base=2, pad=L)
        occ = findall(x -> x == 1, bits)
        if length(occ) == Nf
            mat = U[occ, chosen]
            psi[i+1] = det(mat)
        end
    end
    return psi / norm(psi)
end



In [ ]:
#Testing the functions
N = 10
Nf = 3
H = random_quadratic_H(N)
E, U = diagonalize_H(H)
psi_2 = random_fgs(U, Nf)

# Saving the fermionic gaussian statevector to a text file
open("FGS_Gen_3.txt", "w") do file
    for i in 1:length(psi_2)
        real_part = real(psi_2[i])
        imag_part = imag(psi_2[i])
        write(file, "$real_part + $imag_part im\n")
    end
end